In [6]:
import os
import sys
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

# Configurar GPU si está disponible
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

print("TensorFlow version:", tf.__version__)
print("GPU disponible:", len(physical_devices) > 0)


TensorFlow version: 2.19.0
GPU disponible: False


In [7]:
# Definir la arquitectura MBLLEN
def create_mbllen_model(input_shape=(None, None, 3)):
    input_img = Input(shape=input_shape)
    
    # Branch 1: Procesamiento de características de bajo nivel
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Branch 2: Procesamiento de características de medio nivel
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Branch 3: Procesamiento de características de alto nivel
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    
    # Upsampling y concatenación
    up1 = UpSampling2D(size=(2, 2))(conv3)
    concat1 = Concatenate()([up1, conv2])
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat1)
    
    up2 = UpSampling2D(size=(2, 2))(conv4)
    concat2 = Concatenate()([up2, conv1])
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(concat2)
    
    # Salida
    output = Conv2D(3, (1, 1), activation='sigmoid')(conv5)
    
    model = Model(inputs=input_img, outputs=output)
    return model

# Crear el modelo
model = create_mbllen_model()
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, None,      │        896 │ input_layer_1[0]… │
│                     │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, None,      │          0 │ conv2d_6[0][0]    │
│ (MaxPooling2D)      │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, None,      │     18,496 │ max_pooling2d_2[… │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, None,      │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, None,      │     73,856 │ max_pooling2d_3[… │
│                     │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_2     │ (None, None,      │          0 │ conv2d_8[0][0]    │
│ (UpSampling2D)      │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, None,      │          0 │ up_sampling2d_2[… │
│ (Concatenate)       │ None, 192)        │            │ conv2d_7[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, None,      │    110,656 │ concatenate_2[0]… │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_3     │ (None, None,      │          0 │ conv2d_9[0][0]    │
│ (UpSampling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, None,      │          0 │ up_sampling2d_3[… │
│ (Concatenate)       │ None, 96)         │            │ conv2d_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, None,      │     27,680 │ concatenate_3[0]… │
│                     │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, None,      │         99 │ conv2d_10[0][0]   │
│                     │ None, 3)          │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 231,683 (905.01 KB)

 Trainable params: 231,683 (905.01 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Función para preprocesar la imagen
def preprocess_image(image_path):
    # Leer la imagen
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Normalizar
    img = img.astype(np.float32) / 255.0
    
    # Redimensionar si es necesario
    target_size = (512, 512)  # Puedes ajustar esto según tus necesidades
    if img.shape[0] != target_size[0] or img.shape[1] != target_size[1]:
        img = cv2.resize(img, target_size)
    
    # Expandir dimensiones para el batch
    img = np.expand_dims(img, axis=0)
    return img

# Función para postprocesar la imagen
def postprocess_image(img):
    # Convertir de vuelta a uint8
    img = np.clip(img * 255, 0, 255).astype(np.uint8)
    return img

# Función para procesar una imagen
def process_image(image_path, output_path):
    # Preprocesar
    input_img = preprocess_image(image_path)
    
    # Procesar con el modelo
    enhanced = model.predict(input_img)
    
    # Postprocesar
    result = postprocess_image(enhanced[0])
    
    # Guardar resultado
    cv2.imwrite(output_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
    return output_path


In [9]:
# Ejemplo de uso
if __name__ == "__main__":
    # Rutas de ejemplo
    input_path = "../uploads/users/14/example.png"
    output_path = "../uploads/processed/14/processed_example.png"
    
    # Asegurarse de que el directorio de salida existe
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Procesar la imagen
    try:
        processed_path = process_image(input_path, output_path)
        print(f"Imagen procesada guardada en: {processed_path}")
    except Exception as e:
        print(f"Error al procesar la imagen: {str(e)}")

# Para usar desde Node.js:
# 1. Guardar este notebook como script Python (.py)
# 2. Llamar desde Node.js usando child_process:
"""
const { spawn } = require('child_process');
const processImage = (inputPath, outputPath) => {
    return new Promise((resolve, reject) => {
        const process = spawn('python', ['path/to/script.py', inputPath, outputPath]);
        process.on('close', (code) => {
            if (code !== 0) {
                reject(new Error(`Process exited with code ${code}`));
            } else {
                resolve(outputPath);
            }
        });
    });
};
"""


Error al procesar la imagen: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'



"\nconst { spawn } = require('child_process');\nconst processImage = (inputPath, outputPath) => {\n    return new Promise((resolve, reject) => {\n        const process = spawn('python', ['path/to/script.py', inputPath, outputPath]);\n        process.on('close', (code) => {\n            if (code !== 0) {\n                reject(new Error(`Process exited with code ${code}`));\n            } else {\n                resolve(outputPath);\n            }\n        });\n    });\n};\n"